In [17]:
from transformers import pipeline

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_community.llms import HuggingFacePipeline
import torch
from huggingface_hub import InferenceClient


from pydantic import BaseModel
from typing import List

import os
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
token = os.getenv("HF_TOKEN")

In [14]:
class Gap(BaseModel):
    title: str
    description: str
    impact: str
    actions: List[str]

class GapAnalysis(BaseModel):
    company_name: str
    job_title: str
    generation_date: str
    gaps: List[Gap]

In [15]:
print(torch.mps.is_available())
print(torch.mps.device_count())
print(torch.backends.mps.is_built())



True
1
True


In [ ]:
hf_pipeline = pipeline(
    "text-generation",
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    max_new_tokens=256,
    device='mps',
    dtype=torch.float16
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

Loading weights:  16%|█▌        | 54/339 [00:04<00:23, 12.03it/s, Materializing param=model.layers.4.mlp.up_proj.weight]             


KeyboardInterrupt: 

In [ ]:
print(hf_pipeline.model.device)


mps:0


In [ ]:
parser = PydanticOutputParser(pydantic_object=GapAnalysis)

In [ ]:
prompt = PromptTemplate(
    input_variables=["job_description", "profile"],
    template=
"""
Você é um recrutador técnico.

Vaga:
{job_description}

Perfil do candidato:
{profile}

Tarefa:
Analise a vaga e o perfil do candidato e gere uma análise de gaps retorne em formato JSON.
""",
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    }
)

In [ ]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Gap": {"properties": {"title": {"title": "Title", "type": "string"}, "description": {"title": "Description", "type": "string"}, "impact": {"title": "Impact", "type": "string"}, "actions": {"items": {"type": "string"}, "title": "Actions", "type": "array"}}, "required": ["title", "description", "impact", "actions"], "title": "Gap", "type": "object"}}, "properties": {"company_name": {"title": "Company Name", "type": "string"}, "job_title": {"title": "Job Title", "type": "string"}, "generation_date": {"title": "Generation Date"

In [ ]:
chain = prompt | llm | parser

In [ ]:
with open('./01_jobs/job_1.txt', 'r') as f:

    job_description = f.read()

job_description

'Descrição da vaga\n\nBuscamos profissional com especialidade em aplicar Inteligência Artificial Generativa para integrar nossa equipe de projetos voltados ao atendimento ao cliente. Como integrante do time, será responsável por desenvolver e implementar globalmente soluções inovadoras utilizando principalmente tecnologias de IA generativa, contribuindo diretamente para otimizar nosso negócio e transformar a experiência do cliente através de soluções tecnológicas avançadas.\n\nResponsabilidades e atribuições\n\nDesenvolver de ponta a ponta soluções de negócio utilizando tecnologias de IA generativa e aprendizado de máquina;\nUtilizar ferramentas low/no-code como aceleradores do desenvolvimento e implementação de soluções;\nIdentificar e implementar soluções técnicas otimizadas para atender requisitos específicos de projeto;\nAnalisar sistematicamente oportunidades de melhoria contínua em sistemas já desenvolvidos;\nMaximizar a escalabilidade, desempenho, usabilidade e confiabilidade do

In [ ]:
with open('./02_source/source.txt', 'r') as f:

    source = f.read()

source

'Felipe Castro\nData Scientist | Machine Learning | NLP | Applied AI\nSão Paulo – SP | (11) 93938-8082 | felmateos@alumni.usp.br | linkedin.com/in/felmateos | github.com/felmateos | kaggle.com/felmateos\nEXPERIÊNCIA PROFISSIONAL\nItaú Unibanco 2025 – 2026\nAnalista de Dados São Paulo, SP\n• Desenvolvimento de modelo de Reconhecimento de Entidades Nomeadas (NER) com aprendizado supervisionado e fine-tuning de LLMs.\n• Aplicação em textos corporativos para identificação automática de produtos e volumes, aumentando a rastreabilidade de ofertas.\n• Uso de Python, Transformers e PyTorch em ambiente produtivo.\nItaú Unibanco 2024 – 2025\nEstagiário de Ciência de Dados São Paulo, SP\n• Sistema de OCR com extração automática de entidades e valores em documentos estruturados e semiestruturados.\n• Uso de IA Generativa para substituir processos manuais, reduzindo tempo de processamento e aumentando eficiência operacional.\nUSP – C4AI / IBM / FAPESP 2023 – 2024\nPesquisador Bolsista em Processame

In [ ]:
result = chain.invoke({
    "job_description": job_description,
    "profile": source
})

print(result)

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


: 

: 

In [ ]:
response = client.text_generation(
    prompt,
    max_new_tokens=256,
    temperature=0.0
)
